In [1]:
%pip install -q google-colab-selenium
import google_colab_selenium as gs
driver = gs.Chrome()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 12.8 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
# Install selenium
!pip install selenium

In [3]:
!google-chrome --version


Google Chrome 131.0.6778.204 


In [4]:
!pip install webdriver-manager


In [5]:
!pip install webdriver-manager

In [6]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
from google.colab import drive

# Mount Google Drive
def mount_google_drive():
    try:
        drive.mount('/content/drive')
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        raise

# Check if output directory exists
def check_output_directory(output_directory):
    if not os.path.exists(output_directory):
        print(f"The directory '{output_directory}' does not exist. Please ensure your Google Drive is mounted correctly.")
        raise FileNotFoundError(f"Directory not found: {output_directory}")

# Set up WebDriver
def setup_webdriver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-infobars")
    chrome_options.add_argument("--disable-popup-blocking")
    chrome_options.add_argument("--ignore-certificate-errors")
    chrome_options.add_argument("--incognito")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# Function to scrape discussions
def scrape_huggingface_discussions(driver, url, max_scrolls=10):
    discussions = []
    driver.get(url)
    time.sleep(5)

    scroll_count = 0
    last_height = driver.execute_script("return document.body.scrollHeight")

    while scroll_count < max_scrolls:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.title.raw-link.raw-topic-link'))
        )

        row_elements = driver.find_elements(By.CSS_SELECTOR, 'tr.topic-list-item')

        for row in row_elements:
            try:
                topic_element = row.find_element(By.CSS_SELECTOR, 'a.title.raw-link.raw-topic-link')
                topic_title = topic_element.text.strip()
                topic_link = topic_element.get_attribute('href')

                replies = row.find_element(By.CSS_SELECTOR, 'td.posts span.number').text.strip()
                views = row.find_element(By.CSS_SELECTOR, 'td.views span.number').text.strip()

                driver.execute_script("window.open('{}', '_blank');".format(topic_link))
                driver.switch_to.window(driver.window_handles[-1])
                time.sleep(3)

                try:
                    question_element = driver.find_element(By.CSS_SELECTOR, '.topic-post.clearfix.topic-owner')
                    full_question_text = question_element.text.strip()
                    topic_question = ' '.join(full_question_text.split()[3:])
                    topic_owner = full_question_text.split()[0]

                    try:
                        time_element = question_element.find_element(By.CSS_SELECTOR, 'span.relative-date')
                        created_time = time_element.get_attribute('title')
                    except Exception:
                        created_time = "Timestamp not available"
                except Exception:
                    topic_question = "Question not available"
                    topic_owner = "Unknown"
                    created_time = "Timestamp not available"

                answers = []
                latest_time = "Unknown"
                try:
                    answer_elements = driver.find_elements(By.CSS_SELECTOR, '.topic-post.clearfix.regular')
                    for index, answer_element in enumerate(answer_elements):
                        if index == 0:
                            continue

                        try:
                            username = answer_element.find_element(By.CSS_SELECTOR, 'span.username').text.strip()
                            content = answer_element.find_element(By.CSS_SELECTOR, '.regular .cooked').text.strip()

                            try:
                                time_element = answer_element.find_element(By.CSS_SELECTOR, 'span.relative-date')
                                timestamp = time_element.get_attribute('title')
                                latest_time = timestamp  # Update latest time with each answer
                            except Exception:
                                timestamp = "Timestamp not available"

                            answers.append({
                                "username": username,
                                "content": content,
                                "timestamp": timestamp
                            })
                        except Exception as e:
                            print(f"Error processing answer: {e}")
                            continue
                except Exception as e:
                    print(f"No answers or error occurred: {e}")

                discussions.append({
                    "title": topic_title,
                    "link": topic_link,
                    "owner": topic_owner,
                    "replies": replies,
                    "views": views,
                    "activity": f"Created: {created_time}\nLatest: {latest_time}",
                    "question": topic_question,
                    "time_stamp": created_time,
                    "answers": answers
                })

                driver.close()
                driver.switch_to.window(driver.window_handles[0])
            except Exception as e:
                print(f"Error processing row: {e}")
                continue

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("No more content to load.")
            break
        last_height = new_height
        scroll_count += 1

    return discussions

# Save the scraped data to CSV
def save_to_csv(discussions, file_path):
    data = []
    for discussion in discussions:
        row = {
            "topic_title": discussion['title'],
            "topic_link": discussion['link'],
            "topic_owner": discussion['owner'],
            "replies": discussion['replies'],
            "views": discussion['views'],
            "activity": discussion['activity'],
            "timestamp": discussion['time_stamp'],
            "topic_content": discussion['question'],
        }

        for i, answer in enumerate(discussion['answers'], 1):
            row[f"username_topic_answer_{i}"] = answer['username']
            row[f"topic_answer_{i}"] = answer['content']
            row[f"timestamp_topic_answer_{i}"] = answer['timestamp']

        data.append(row)

    df = pd.DataFrame(data)
    df.to_csv(file_path, index=False)
    print(f"Data saved to {file_path}")

# Main Execution
if __name__ == "__main__":
    mount_google_drive()
    output_directory = '/content/drive/My Drive'
    check_output_directory(output_directory)

    driver = setup_webdriver()
    url = "https://discuss.huggingface.co/c/beginners/5/l/hot"

    try:
        discussions = scrape_huggingface_discussions(driver, url)
        file_path = os.path.join(output_directory, 'scraped_discussions.csv')  
        save_to_csv(discussions, file_path)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.quit()

    print("Web scraping and file saving completed successfully!")


Mounted at /content/drive
Data saved to /content/drive/My Drive/scraped_discussions.xlsx
Web scraping and file saving completed successfully!
